## Prepare neural network and data loaders:

In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Get MNIST data:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True,
                                           transform=transforms.ToTensor())
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True,
                                          transform=transforms.ToTensor())

# Create data loaders for batch training: 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True) # Randomize training data order
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Build the nn model as a class/object:
class Net(nn.Module): # Class Net inherits from the nn.Module class
    
    def __init__(self): # Specify architecture
        super(Net, self).__init__()
        self.layer1 = nn.Linear(28*28, 128) # Input* 28*28 -> 128 hidden nodes
        self.layer2 = nn.Linear(128, 64) # 128 hidden nodes -> 64 hidden nodes
        self.output = nn.Linear(64, 10) # Output: hidden nodes 64 -> 10 labels

    def forward(self, x): # Specify activation
        x = x.view(-1, 28*28) # Input layer
        x = F.relu(self.layer1(x)) # Relu layer 1
        x = F.relu(self.layer2(x)) # Relu layer 2
        x = self.output(x) # Output layer
        return x

model = Net() # Initialize untrained model

# Select loss function and optimizer: 
criterion = nn.CrossEntropyLoss() # Loss function: Categorical cross entropy
optimizer = optim.Adam(model.parameters()) # Optimizer: Adam
L = 15 # training length (another hyperparameter)


100%|██████████| 9.91M/9.91M [00:00<00:00, 23.5MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 5.24MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 17.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 17.0MB/s]


## Train model:

In [3]:
# Train model: 
for epoch in range(L): # Control length of training
    for i, data in enumerate(train_loader): 
        inputs, labels = data # Get this data chunk
        optimizer.zero_grad() # Set gradients to zero
        outputs = model(inputs) # Use current model to make predictions 
        loss = criterion(outputs, labels) # Evaluate current loss
        loss.backward() # Back-propagation for next iteration
        optimizer.step() # Find a lower loss set of weights

## Evaluate model:

In [4]:
y_hat = [] # Predicted values
y_act = [] # Actual values

with torch.no_grad(): # Do not update gradient
    for data in test_loader: # Iterate over data batches
        inputs, labels = data # (y,X)
        outputs = model(inputs) # evaluate model for these data
        max_values, predicted_values = torch.max(outputs.data, 1) # extract predictions
        y_act+=labels.tolist() # Add actual values
        y_hat+=predicted_values.tolist() # Add predicted values

print( pd.crosstab(y_hat, y_act) ) # Confusion matrix

acc = np.sum( [y_hat[i] == y_act[i] for i in range(len(y_hat))] ) / len(y_hat)
print('Accuracy: %d %%' % (100 * acc)) # Accuracy

col_0    0     1     2    3    4    5    6     7    8    9
row_0                                                     
0      970     0     2    0    1    3    2     2    6    2
1        0  1123     0    0    0    1    1     2    1    3
2        2     3  1016    7    3    0    0     8    3    1
3        1     1     1  989    0   14    1     3   12    6
4        1     0     1    0  958    1    3     0    4    7
5        0     0     0    4    1  864    2     2    5    2
6        3     2     2    0    5    6  949     0    3    3
7        1     2     6    6    6    1    0  1008    6    7
8        2     4     4    2    2    2    0     0  932    3
9        0     0     0    2    6    0    0     3    2  975
Accuracy: 97 %
